In [1]:
import mujoco
import glfw
import numpy as np
import time
from PIL import Image

XML_FILE_PATH = "/Users/edoardozappia/Desktop/Tesi_Magistrale/irregular_shape_2D.xml"
SAVE_PATH = "/Users/edoardozappia/Desktop/Tesi_Magistrale/target_positions_semicircle.npy"

def main():
    R = 400  # Raggio massimo della circonferenza
    omega = np.pi / 2  # Velocità angolare costante per il semicerchio
    dt = 0.01  # Timestep fisso
    noise = 0.0  # Rumore bianco gaussiano
    mass = 1.0  # Massa ipotetica per il calcolo della forza
    start_angle = 0  # Inizio del semicerchio
    end_angle = np.pi  # Fine del semicerchio
    theta = start_angle  # Angolo iniziale

    if not glfw.init():
        print("Impossibile inizializzare GLFW")
        return

    glfw.window_hint(glfw.VISIBLE, glfw.TRUE)
    window = glfw.create_window(800, 800, "MuJoCo Viewer", None, None)
    if not window:
        print("Errore durante la creazione della finestra GLFW")
        glfw.terminate()
        return

    glfw.make_context_current(window)

    model = mujoco.MjModel.from_xml_path(XML_FILE_PATH)
    data = mujoco.MjData(model)

    # Configura la telecamera
    camera = mujoco.MjvCamera()
    camera.type = mujoco.mjtCamera.mjCAMERA_FREE
    camera.lookat = np.array([0, 0, 0])
    camera.distance = 2.0
    camera.azimuth = 90
    camera.elevation = -90

    # Renderer per visualizzare la finestra
    scene = mujoco.MjvScene(model, maxgeom=1000)
    context = mujoco.MjrContext(model, mujoco.mjtFontScale.mjFONTSCALE_150)

    t = 0  # Tempo iniziale
    positions = []

    while not glfw.window_should_close(window) and theta <= end_angle:
        t += dt
        theta += omega * dt  # Incrementa l'angolo per mantenere la traiettoria semicircolare

        # Posizione attuale
        x = data.qpos[0]
        y = data.qpos[1]

        # Calcolo della forza per mantenere la traiettoria semicircolare
        force_x = -mass * R * (omega ** 2) * np.cos(theta) + noise * np.random.randn()
        force_y = -mass * R * (omega ** 2) * np.sin(theta) + noise * np.random.randn()

        torque_phi = 0  # Nessuna rotazione necessaria per il semicerchio

        # Salva la posizione corrente
        positions.append([x, y])

        # Applica le forze
        data.qfrc_applied[0] = force_x
        data.qfrc_applied[1] = force_y
        data.qfrc_applied[2] = torque_phi

        # Avanza la simulazione
        mujoco.mj_step(model, data)

        # Debug
        print(f"Posizione: x={x:.3f}, y={y:.3f}")
        print(f"Forze: force_x={force_x:.3f}, force_y={force_y:.3f}")

        # Renderizza la scena
        mujoco.mjv_updateScene(model, data, mujoco.MjvOption(), None, camera, mujoco.mjtCatBit.mjCAT_ALL, scene)
        viewport_width, viewport_height = glfw.get_framebuffer_size(window)
        viewport = mujoco.MjrRect(0, 0, viewport_width, viewport_height)
        mujoco.mjr_render(viewport, scene, context)

        glfw.swap_buffers(window)
        glfw.poll_events()
        time.sleep(dt)

    # Salva il dataset delle posizioni target
    positions = np.array(positions)
    print(f"Steps: {len(positions)}")
    np.save(SAVE_PATH, positions)
    print(f"Dataset delle posizioni target salvato in: {SAVE_PATH}")

    glfw.destroy_window(window)
    glfw.terminate()

if __name__ == "__main__":
    main()


Posizione: x=0.000, y=0.000
Forze: force_x=-986.839, force_y=-15.503
Posizione: x=-0.002, y=-0.000
Forze: force_x=-986.473, force_y=-31.001
Posizione: x=-0.003, y=-0.000
Forze: force_x=-985.865, force_y=-46.492
Posizione: x=-0.005, y=-0.000
Forze: force_x=-985.013, force_y=-61.972
Posizione: x=-0.007, y=-0.000
Forze: force_x=-983.918, force_y=-77.436
Posizione: x=-0.008, y=-0.000
Forze: force_x=-982.580, force_y=-92.881
Posizione: x=-0.010, y=-0.001
Forze: force_x=-981.000, force_y=-108.303
Posizione: x=-0.011, y=-0.001
Forze: force_x=-979.178, force_y=-123.699
Posizione: x=-0.013, y=-0.001
Forze: force_x=-977.114, force_y=-139.064
Posizione: x=-0.015, y=-0.001
Forze: force_x=-974.809, force_y=-154.395
Posizione: x=-0.016, y=-0.001
Forze: force_x=-972.264, force_y=-169.687
Posizione: x=-0.018, y=-0.002
Forze: force_x=-969.479, force_y=-184.938
Posizione: x=-0.019, y=-0.002
Forze: force_x=-966.454, force_y=-200.143
Posizione: x=-0.021, y=-0.002
Forze: force_x=-963.191, force_y=-215.299
